# MNC-KAIST
## (1) Neural Network
### 2021 / 08 / 31
Copyright @ cb_park@korea.ac.kr (Cheonbok Park), joonleesky@kaist.ac.kr (Hojoon Lee)

## MNIST Feed-Forward Neural Network

In [1]:
import torch 
import torch.nn as nn #
import torch.nn.functional as F # 각종 activation 함수
import torchvision # 이미지 관련 처리, Pretrained Model 관련된 Package 입니다. 
import torchvision.datasets as vision_dsets
import torchvision.transforms as T # 이미지 처리 (Vison) 관련된 transformation이 정의 되어 있습니다.
import torch.optim as optim # pytorch 에서 정의한 수 많은 optimization function 들이 들어 있습니다.
from torch.utils import data

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

## Data Loader 불러오기

In [2]:
def MNIST_DATA(root='./data',train =True,transforms=None ,download =True,batch_size = 32,num_worker = 1):
    print ("[+] Get the MNIST DATA")
    """
    torchvision.dataset 에는 우리가 많이 사용하는 데이터들을 쉽게 사용할 수 있도록 되어 있습니다. 
    Mchine Learning 에서 Hello world 라고 불리는 Mnist 데이터를 사용해 보겠습니다. 
    """
    mnist_train = vision_dsets.MNIST(root = root,  #root 는 데이터의 저장 위치 입니다. 
                                    train = True, #Train 은 이 데이터가 train 데이터인지 아닌지에 대한 정보입니다. 
                                    transform = T.ToTensor(), # 얻어낸 데이터를 pytorch가 계산 할 수 있는 Tensor 로 변환해 줍니다. 
                                    download = True)  # 데이터를 다운로드 할지 여부를 물어봅니다. 
    mnist_test = vision_dsets.MNIST(root = root,
                                    train = False,  # Test Data를 가져오기에 Train =False 를 줘야 합니다. 
                                    transform = T.ToTensor(),
                                    download = True)
    """
    Data Loader 는 데이터와 batch size의 정보를 바탕으로 매 iteration 마다 주어진 데이터를 원하는 batch size 만큼 반환해주는 iterator입니다. 
    * Practical Guide : Batch size 는 어느정도가 좋나요? -- 클 수록 좋다는 소리가 있습니다. 하지만 gpu memeory 사이즈 한계에 의해 기본적으로 batch size 가 
      커질 수록 학습에 사용되는 gpu memory 사이즈가 큽니다. (Activation map을 저장해야 하기 때문입니다.) 기본적으로 2의 배수로 저장하는 것이 좋습니다.(Bit size 관련) 
    """
    trainDataLoader = data.DataLoader(dataset = mnist_train,  # DataSet은 어떤 Data를 제공해 줄지에 대한 정보입니다. 여기서는 Training DATA를 제공합니다. 
                                      batch_size = batch_size, # batch size 정보를 꼭 줘야 합니다. 한 Batch 당 몇 개의 Data 를 제공할지에 대한 정보입니다. 
                                      shuffle =True, # Training의 경우 Shuffling 을 해주는 것이 성능에 지대한 영향을 끼칩니다. 꼭 True 를 줘야 합니다. 
                                      num_workers = 1) # num worker의 경우 데이터를 로드하는데 worker를 얼마나 추가하겠는가에 대한 정보입니다. 

    testDataLoader = data.DataLoader(dataset = mnist_test, # Test Data Loader 이므로 Test Data를 인자로 전달해줍니다.
                                    batch_size = batch_size, # 마찬가지로 Batch size 를 넣어줍니다. 
                                    shuffle = False, # shuffling 이 굳이 필요하지 않으므로 false를 줍니다. 
                                    num_workers = 1) #
    print ("[+] Finished loading data & Preprocessing")
    return mnist_train,mnist_test,trainDataLoader,testDataLoader

In [3]:
trainDset,testDset,trainDataLoader,testDataLoader= MNIST_DATA(batch_size = 32)  # Data Loader 를 불러 옵니다. 

[+] Get the MNIST DATA
[+] Finished loading data & Preprocessing


## Trainer 정의하기

In [11]:
class Trainer():
    def __init__(self, trainloader, testloader, net, optimizer, criterion):
        """
        trainloader: train data의 loader 입니다
        testloader: test data의 loader 입니다
        net: 학습시킬 모델입니다
        optimizer: 모델의 파라미터를 업데이트할 최적화 함수입니다
        criterion: 모델의 loss function 입니다.
        """
        self.trainloader = trainloader
        self.testloader = testloader
        self.net = net
        self.optimizer = optimizer
        self.criterion = criterion
        
    def train(self, epoch = 1):
        """
        epoch: 전체 학습 데이터의 사용횟수입니다.
        """
        self.net.train()
        for e in range(epoch):
            running_loss = 0.0 # running loss를 저장하기 위한 변수입니다. 
            for i, data in enumerate(self.trainloader, 0): # 한 Epoch 만큼 돕니다. 매 iteration 마다 정해진 Batch size 만큼 데이터를 뱉습니다. 
                # get the inputs
                inputs, labels = data # DataLoader iterator의 반환 값은 input_data 와 labels의 튜플 형식입니다. 
                inputs = inputs
                labels = labels
                # zero the parameter gradients
                self.optimizer.zero_grad()    #  현재 기존의 backprop을 계산하기 위해서 저장했던 activation buffer 를 비웁니다. 
                                              #  Q) 이걸 안 한다면?

                # forward + backward + optimize
                outputs = self.net(inputs) # input 을 넣은 위 network 로 부터 output 을 얻어냅니다. 
                loss = self.criterion(outputs, labels) # loss fucntion에 주어진 target과 output 의 score를 계산하여 반환합니다. 
                loss.backward() # * Scalar Loss value를 Backward() 해주게 되면 주어진 loss값을 바탕으로 backpropagation이 진행됩니다. 
                self.optimizer.step() # 계산된 Backprop 을 바탕으로 optimizer가 gradient descenting 을 수행합니다. 

                # print statistics
                running_loss += loss.item()
                if (i+1) % 500 == 0:    # print every 2000 mini-batches
                    print('[%d, %5d] loss: %.3f' % (e + 1, i + 1, running_loss / 500))
                    running_loss = 0.0

        print('Finished Training')
        
    def test(self):
        self.net.eval() # Eval Mode 왜 해야 할까요?  
                        # --> nn.Dropout BatchNorm 등의 Regularization 들이 test 모드로 들어가게 되기 때문입니다. 
        test_loss = 0
        correct = 0
        for inputs, labels in self.testloader:
            inputs = inputs
            labels = labels 
            output = self.net(inputs) 
            pred = output.max(1, keepdim=True)[1] # get the index of the max 
            correct += pred.eq(labels.view_as(pred)).sum().item() # 정답 데이터의 갯수를 반환합니다. 

            test_loss /= len(self.testloader.dataset)
        print('\nTest set:  Accuracy: {}/{} ({:.0f}%)\n'.
                format(correct, len(self.testloader.dataset),
                100.* correct / len(self.testloader.dataset)))

## Network 만들어보기

![activation](./imgs/activation.png)
출처: cs231n

### (1) 2-Layer Network + Sigmoid

- Input: (28 * 28)
- Hidden dimension: 30
- Output dimension: 10
- activation: sigmoid
- Optimizer: SGD
- Loss: Cross-Entropy

In [12]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__() # nn.Module 생성자 호출 Q) 왜 필요할까요?
        # an affine operation: y = Wx + b
        self.fc0 = nn.Linear(28*28,30)
        self.fc1 = nn.Linear(30, 10)

    def forward(self, x):
        x = x.view(-1,28*28) # x.view함수는 주어진 인자의 크기로 해당 데이터의 크기를 반환합니다. 즉, (Batch_size,28,28) --> (Batch_size,28*28)로 변환합니다.
        x = self.fc0(x) # 28*28 -> 30 
        x = F.sigmoid(x) # Activation function 을 수행합니다.
        x = self.fc1(x)  # 30 -> 10 으로 10개의 Class에 대한 logit 값을 호출합니다. 
        return x

In [13]:
mnist_net = MNIST_Net() # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다.
# mnist_net = MNIST_Net().cuda()
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.SGD(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [14]:
trainer = Trainer(trainloader = trainDataLoader,
                  testloader = testDataLoader,
                  net = mnist_net,
                  criterion = criterion,
                  optimizer = optimizer)

In [15]:
trainer.train(epoch = 4)

C:\Users\chahn\anaconda3\lib\site-packages\torch\nn\functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[1,   500] loss: 2.337
[1,  1000] loss: 2.293
[1,  1500] loss: 2.273
[2,   500] loss: 2.244
[2,  1000] loss: 2.231
[2,  1500] loss: 2.217
[3,   500] loss: 2.194
[3,  1000] loss: 2.174
[3,  1500] loss: 2.161
[4,   500] loss: 2.134
[4,  1000] loss: 2.114
[4,  1500] loss: 2.095
Finished Training


In [16]:
trainer.test()


Test set:  Accuracy: 5292/10000 (53%)



### (2) 2-Layer Network + ReLU

- Input: (28 * 28)
- Hidden dimension: 30
- Output dimension: 10
- activation: relu
- Optimizer: SGD
- Loss: Cross-Entropy

In [29]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__() # nn.Module 생성자 호출 Q) 왜 필요할까요?
        # an affine operation: y = Wx + b
        self.fc0 = nn.Linear(28*28,30)
        self.fc1 = nn.Linear(30, 10)

    def forward(self, x):
        x = x.view(-1,28*28) # x.view함수는 주어진 인자의 크기로 해당 데이터의 크기를 반환합니다. 즉, (Batch_size,28,28) --> (Batch_size,28*28)로 변환합니다.
        x = self.fc0(x) # 28*28 -> 30 
        x = F.relu(x) # Activation function 을 수행합니다.
        x = self.fc1(x)  # 30 -> 10 으로 10개의 Class에 대한 logit 값을 호출합니다. 
        return x

In [30]:
mnist_net = MNIST_Net() # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.SGD(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [31]:
trainer = Trainer(trainloader = trainDataLoader,
                  testloader = testDataLoader,
                  net = mnist_net,
                  criterion = criterion,
                  optimizer = optimizer)

In [32]:
trainer.train(epoch = 4)

[1,   500] loss: 2.277
[1,  1000] loss: 2.187
[1,  1500] loss: 2.085
[2,   500] loss: 1.876
[2,  1000] loss: 1.734
[2,  1500] loss: 1.589
[3,   500] loss: 1.352
[3,  1000] loss: 1.224
[3,  1500] loss: 1.123
[4,   500] loss: 0.976
[4,  1000] loss: 0.907
[4,  1500] loss: 0.847
Finished Training


In [33]:
trainer.test()


Test set:  Accuracy: 8345/10000 (83%)



#### Q) Activation별로 성능차이가 존재하나요? 존재한다면 왜 존재할까요?

#### Ans)

### (3) 3-Layer Network + Sigmoid

- Input: (28 * 28)
- Hidden dimension: (50, 30)
- Output dimension: 10
- activation: sigmoid
- Optimizer: SGD
- Loss: Cross-Entropy

In [34]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__()
        self.fc0 = nn.Linear(28*28,50) # Layer 1
        self.fc1 = nn.Linear(50, 30) # Layer 2
        self.fc2 = nn.Linear(30, 10) # Layer 3

    def forward(self, x):
      
        x = x.view(-1,28*28)
        x = self.fc0(x)
        x = F.sigmoid(x)
        x = self.fc1(x)
        x = F.sigmoid(x)
        x = self.fc2(x)
        return x

In [36]:
mnist_net = MNIST_Net() # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.SGD(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [37]:
trainer = Trainer(trainloader = trainDataLoader,
                  testloader = testDataLoader,
                  net = mnist_net,
                  criterion = criterion,
                  optimizer = optimizer)

In [38]:
trainer.train(epoch = 4)

[1,   500] loss: 2.346
[1,  1000] loss: 2.320
[1,  1500] loss: 2.307
[2,   500] loss: 2.301
[2,  1000] loss: 2.300
[2,  1500] loss: 2.300
[3,   500] loss: 2.298
[3,  1000] loss: 2.299
[3,  1500] loss: 2.299
[4,   500] loss: 2.297
[4,  1000] loss: 2.298
[4,  1500] loss: 2.298
Finished Training


In [39]:
trainer.test()


Test set:  Accuracy: 1135/10000 (11%)



#### Q) 학습이 원할하게 이루어지나요? 이루어지지 않는다면 왜 이루어지지 않을까요?

#### Ans)

### (4) 3-Layer Network + ReLU

- Input: (28 * 28)
- Hidden dimension: (50, 30)
- Output dimension: 10
- activation: relu
- Optimizer: SGD
- Loss: Cross-Entropy

In [40]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__()
        self.fc0 = nn.Linear(28*28,50) # Layer 1
        self.fc1 = nn.Linear(50, 30) # Layer 2
        self.fc2 = nn.Linear(30, 10) # Layer 3

    def forward(self, x):
      
        x = x.view(-1,28*28)
        x = self.fc0(x)
        x = F.relu(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

In [41]:
mnist_net = MNIST_Net() # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.SGD(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [42]:
trainer = Trainer(trainloader = trainDataLoader,
                  testloader = testDataLoader,
                  net = mnist_net,
                  criterion = criterion,
                  optimizer = optimizer)

In [43]:
trainer.train(epoch = 4)

[1,   500] loss: 2.303
[1,  1000] loss: 2.292
[1,  1500] loss: 2.277
[2,   500] loss: 2.245
[2,  1000] loss: 2.219
[2,  1500] loss: 2.186
[3,   500] loss: 2.099
[3,  1000] loss: 2.032
[3,  1500] loss: 1.949
[4,   500] loss: 1.753
[4,  1000] loss: 1.626
[4,  1500] loss: 1.489
Finished Training


In [44]:
trainer.test()


Test set:  Accuracy: 7043/10000 (70%)



#### Q) (2)와 비교했을 때 학습이 원할하게 이루어지나요? 이루어지지 않는다면 왜 이루어지지 않을까요?

#### Ans)

#### Q) Activation을 사용하지 않는다면 어떤 일이 일어날까요?

#### Ans) 

### (5) 3-Layer Network + Sine

- Input: (28 * 28)
- Hidden dimension: (50, 30)
- Output dimension: 10
- activation: Sine
- Optimizer: SGD
- Loss: Cross-Entropy

In [45]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__()
        self.fc0 = nn.Linear(28*28,50) # Layer 1
        self.fc1 = nn.Linear(50, 30) # Layer 2
        self.fc2 = nn.Linear(30, 10) # Layer 3

    def forward(self, x):
      
        x = x.view(-1,28*28)
        x = self.fc0(x)
        x = torch.sin(x)
        x = self.fc1(x)
        x = torch.sin(x)
        x = self.fc2(x)
        return x

In [46]:
mnist_net = MNIST_Net() # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.SGD(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [47]:
trainer = Trainer(trainloader = trainDataLoader,
                  testloader = testDataLoader,
                  net = mnist_net,
                  criterion = criterion,
                  optimizer = optimizer)

In [48]:
trainer.train(epoch = 4)

[1,   500] loss: 2.277
[1,  1000] loss: 2.212
[1,  1500] loss: 2.140
[2,   500] loss: 1.986
[2,  1000] loss: 1.875
[2,  1500] loss: 1.750
[3,   500] loss: 1.530
[3,  1000] loss: 1.400
[3,  1500] loss: 1.306
[4,   500] loss: 1.151
[4,  1000] loss: 1.085
[4,  1500] loss: 1.007
Finished Training


In [49]:
trainer.test()


Test set:  Accuracy: 8018/10000 (80%)



#### Q) (2)와 비교했을 때 학습이 원할하게 이루어지나요? 이루어지지 않는다면 왜 이루어지지 않을까요?

#### Ans)

## Let's Change our Optimizer

![Adam](./imgs/adam.jpeg)
출처: 하용호, 자습해도 모르겠던 딥러닝, 머리속에 인스톨 시켜드립니다

### (6) 3-Layer Network + ReLU + Adam

- Input: (28 * 28)
- Hidden dimension: (50, 30)
- Output dimension: 10
- activation: relu
- Optimizer: Adam
- Loss: Cross-Entropy

In [199]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__()
        self.fc0 = nn.Linear(28*28,50) # Layer 1
        self.fc1 = nn.Linear(50, 30) # Layer 2
        self.fc2 = nn.Linear(30, 10) # Layer 3

    def forward(self, x):
      
        x = x.view(-1,28*28)
        x = self.fc0(x)
        x = F.relu(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

In [200]:
mnist_net = MNIST_Net() # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.Adam(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [201]:
trainer = Trainer(trainloader = trainDataLoader,
                  testloader = testDataLoader,
                  net = mnist_net,
                  criterion = criterion,
                  optimizer = optimizer)

In [204]:
trainer.train(epoch = 5)

[1,   500] loss: 0.102
[1,  1000] loss: 0.109
[1,  1500] loss: 0.104
[2,   500] loss: 0.086
[2,  1000] loss: 0.085
[2,  1500] loss: 0.094
[3,   500] loss: 0.073
[3,  1000] loss: 0.074
[3,  1500] loss: 0.075
[4,   500] loss: 0.063
[4,  1000] loss: 0.065
[4,  1500] loss: 0.069
[5,   500] loss: 0.055
[5,  1000] loss: 0.056
[5,  1500] loss: 0.054
Finished Training


In [205]:
trainer.test()


Test set:  Accuracy: 9695/10000 (97%)



### (7) 2-Layer Network + ReLU + Adam

- Input: (28 * 28)
- Hidden dimension: (30)
- Output dimension: 10
- activation: relu
- Optimizer: Adam
- Loss: Cross-Entropy

In [55]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__() # nn.Module 생성자 호출 Q) 왜 필요할까요?
        # an affine operation: y = Wx + b
        self.fc0 = nn.Linear(28*28,30)
        self.fc1 = nn.Linear(30, 10)

    def forward(self, x):
        x = x.view(-1,28*28) # x.view함수는 주어진 인자의 크기로 해당 데이터의 크기를 반환합니다. 즉, (Batch_size,28,28) --> (Batch_size,28*28)로 변환합니다.
        x = self.fc0(x) # 28*28 -> 30 
        x = F.relu(x) # Activation function 을 수행합니다.
        x = self.fc1(x)  # 30 -> 10 으로 10개의 Class에 대한 logit 값을 호출합니다. 
        return x

In [56]:
mnist_net = MNIST_Net() # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.Adam(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [57]:
trainer = Trainer(trainloader = trainDataLoader,
                  testloader = testDataLoader,
                  net = mnist_net,
                  criterion = criterion,
                  optimizer = optimizer)

In [58]:
trainer.train(epoch = 4)

[1,   500] loss: 0.681
[1,  1000] loss: 0.327
[1,  1500] loss: 0.289
[2,   500] loss: 0.231
[2,  1000] loss: 0.219
[2,  1500] loss: 0.214
[3,   500] loss: 0.180
[3,  1000] loss: 0.187
[3,  1500] loss: 0.170
[4,   500] loss: 0.152
[4,  1000] loss: 0.151
[4,  1500] loss: 0.149
Finished Training


In [59]:
trainer.test()


Test set:  Accuracy: 9535/10000 (95%)



## Batch-Normalization

![normalization](./imgs/normalization.png)
출처: Andrew Ng, Deep Learning

### (8) 2-Layer Network + ReLU + Adam + Batch-Norm

- Input: (28 * 28)
- Hidden dimension: (30)
- Output dimension: 10
- activation: relu
- normalization: batch-norm
- Optimizer: Adam
- Loss: Cross-Entropy

In [60]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__() # nn.Module 생성자 호출 Q) 왜 필요할까요?
        # an affine operation: y = Wx + b
        self.fc0 = nn.Linear(28*28,30)
        self.bn0 = nn.BatchNorm1d(30) # BatchNorm 
        self.fc1 = nn.Linear(30, 10)

    def forward(self, x):
        x = x.view(-1,28*28) # x.view함수는 주어진 인자의 크기로 해당 데이터의 크기를 반환합니다. 즉, (Batch_size,28,28) --> (Batch_size,28*28)로 변환합니다.
        x = self.fc0(x) # 28*28 -> 30 
        x = self.bn0(x)
        x = F.relu(x) # Activation function 을 수행합니다.
        x = self.fc1(x)  # 30 -> 10 으로 10개의 Class에 대한 logit 값을 호출합니다. 
        return x

In [61]:
mnist_net = MNIST_Net() # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.Adam(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [62]:
trainer = Trainer(trainloader = trainDataLoader,
                  testloader = testDataLoader,
                  net = mnist_net,
                  criterion = criterion,
                  optimizer = optimizer)

In [63]:
trainer.train(epoch = 4)

[1,   500] loss: 0.705
[1,  1000] loss: 0.337
[1,  1500] loss: 0.285
[2,   500] loss: 0.215
[2,  1000] loss: 0.218
[2,  1500] loss: 0.197
[3,   500] loss: 0.169
[3,  1000] loss: 0.171
[3,  1500] loss: 0.171
[4,   500] loss: 0.152
[4,  1000] loss: 0.147
[4,  1500] loss: 0.159
Finished Training


In [65]:
trainer.test()


Test set:  Accuracy: 9621/10000 (96%)



### (9) 3-Layer Network + ReLU + Adam + Batch-Norm

- Input: (28 * 28)
- Hidden dimension: (50, 30)
- Output dimension: 10
- activation: relu
- normalization: batch-norm
- Optimizer: Adam
- Loss: Cross-Entropy

In [66]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__()
        self.fc0 = nn.Linear(28*28,50) # Layer 1
        self.bn0 = nn.BatchNorm1d(50) # BatchNorm 1 
        self.fc1 = nn.Linear(50, 30) # Layer 2
        self.bn1 = nn.BatchNorm1d(30) # BatchNorm 2
        self.fc2 = nn.Linear(30, 10) # Layer 3

    def forward(self, x):
      
        x = x.view(-1,28*28)
        x = self.fc0(x)
        x = self.bn0(x)
        x = F.relu(x)
        x = self.fc1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

In [67]:
mnist_net = MNIST_Net() # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.Adam(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [68]:
trainer = Trainer(trainloader = trainDataLoader,
                  testloader = testDataLoader,
                  net = mnist_net,
                  criterion = criterion,
                  optimizer = optimizer)

In [69]:
trainer.train(epoch = 4)

[1,   500] loss: 0.620
[1,  1000] loss: 0.244
[1,  1500] loss: 0.203
[2,   500] loss: 0.153
[2,  1000] loss: 0.140
[2,  1500] loss: 0.140
[3,   500] loss: 0.115
[3,  1000] loss: 0.115
[3,  1500] loss: 0.122
[4,   500] loss: 0.099
[4,  1000] loss: 0.092
[4,  1500] loss: 0.104
Finished Training


In [70]:
trainer.test()


Test set:  Accuracy: 9725/10000 (97%)



#### Q) Batch-norm을 적용하기 전과 후의 성능이 어떤가요? 

#### Ans)

#### Q) Batch-norm을 적용한 이후의 2-layer와 3-layer network의 성능은 어떤 변화 양상을 보여주었나요?

#### Ans)

In [71]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_parameters(mnist_net)

41250

#### Q) 파라미터 수가 많으면 어떠한 문제점이 발생할까요?

#### Ans)

## Let's use Convolution Layer

### Convolution Operation

![Convolution](./imgs/Conv.png)

#### Q) (H, W, C1)의 이미지에 C2개의 (F * F) filter를 stride 크기 S로 convolution하면 output size가 어떻게 될까요?
#### Ans)

### (10) 3-Layer Network (Conv+Fc) + ReLU + Adam + Batch-Norm

- Input: (28 * 28)
- Conv: 8 (6 * 6) filter with stride=2 
- Hidden dimension: 8 * 12 * 12
- Output dimension: 10
- activation: relu
- normalization: batch-norm
- Optimizer: Adam
- Loss: Cross-Entropy

In [184]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__()
        self.conv0 = nn.Conv2d(in_channels = 1,
                               out_channels = 8,
                               kernel_size = 6,
                               stride = 2) # Layer 1
        self.conv0_bn = nn.BatchNorm2d(8)  # Image에서는 2d batchnorm이 사용됩니다
        self.fc = nn.Linear(8*12*12, 10) # Layer 2 (왜 hidden이 8 * 12 * 12 일까요?)

    def forward(self, x):
        x = self.conv0(x)
        x = self.conv0_bn(x)
        x = F.relu(x)
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        return x

In [185]:
mnist_net = MNIST_Net() # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.Adam(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [186]:
trainer = Trainer(trainloader = trainDataLoader,
                  testloader = testDataLoader,
                  net = mnist_net,
                  criterion = criterion,
                  optimizer = optimizer)

In [187]:
trainer.train(epoch = 5)

[1,   500] loss: 0.400
[1,  1000] loss: 0.176
[1,  1500] loss: 0.133
[2,   500] loss: 0.089
[2,  1000] loss: 0.091
[2,  1500] loss: 0.080
[3,   500] loss: 0.064
[3,  1000] loss: 0.066
[3,  1500] loss: 0.060
[4,   500] loss: 0.051
[4,  1000] loss: 0.053
[4,  1500] loss: 0.050
[5,   500] loss: 0.046
[5,  1000] loss: 0.044
[5,  1500] loss: 0.047
Finished Training


In [188]:
trainer.test()


Test set:  Accuracy: 9796/10000 (98%)



In [80]:
count_parameters(mnist_net)

11842

#### Q) Convolution operation을 사용했을 때 성능과 파라미터가 어떻게 변했나요? 왜 이런 결과가 나왔을까요?

#### Ans)

### (11) 3-Layer Network (Conv+Pool+Fc) + ReLU + Adam + Batch-Norm

- Input: (28 * 28)
- Conv: 8 (7 * 7) filter with stride=2 
- Pool: 2 * 2
- Hidden dimension: 8 * 6 * 6
- Output dimension: 10
- activation: relu
- normalization: batch-norm
- Optimizer: Adam
- Loss: Cross-Entropy

### Pooling Operation

![Pooling](./imgs/Pool.png)

In [81]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__()
        self.conv0 = nn.Conv2d(in_channels = 1,
                               out_channels = 8,
                               kernel_size = 6,
                               stride = 2) # Layer 1
        self.conv0_bn = nn.BatchNorm2d(8)  # Image에서는 2d batchnorm이 사용됩니다
        self.pool0 = nn.MaxPool2d(2)
        self.fc = nn.Linear(8*6*6, 10) # Layer 2 (왜 input이 8 * 11 * 11 일까요?)

    def forward(self, x):
        x = self.conv0(x)
        x = self.conv0_bn(x)
        x = F.relu(x)
        x = self.pool0(x)
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        return x

In [82]:
mnist_net = MNIST_Net() # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.Adam(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [83]:
trainer = Trainer(trainloader = trainDataLoader,
                  testloader = testDataLoader,
                  net = mnist_net,
                  criterion = criterion,
                  optimizer = optimizer)

In [84]:
trainer.train(epoch = 4)

[1,   500] loss: 0.515
[1,  1000] loss: 0.192
[1,  1500] loss: 0.145
[2,   500] loss: 0.106
[2,  1000] loss: 0.104
[2,  1500] loss: 0.090
[3,   500] loss: 0.080
[3,  1000] loss: 0.077
[3,  1500] loss: 0.079
[4,   500] loss: 0.069
[4,  1000] loss: 0.073
[4,  1500] loss: 0.066
Finished Training


In [85]:
trainer.test()


Test set:  Accuracy: 9772/10000 (98%)



In [86]:
count_parameters(mnist_net)

3202

#### Q) Pooling을 쓰기 전과 후에 성능과 파라미터 수가 어떻게 변했나요? 왜 이런 결과가 나왔을까요?

#### Ans)

### (12) 3-Layer Network (Conv+Pool+Fc) + ReLU + Adam + Instance-Norm

- Input: (28 * 28)
- Conv: 8 (7 * 7) filter with stride=2 
- Pool: 2 * 2
- Hidden dimension: 8 * 6 * 6
- Output dimension: 10
- activation: relu
- normalization: Instance-norm
- Optimizer: Adam
- Loss: Cross-Entropy

In [87]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__()
        self.conv0 = nn.Conv2d(in_channels = 1,
                               out_channels = 8,
                               kernel_size = 6,
                               stride = 2) # Layer 1
        self.conv0_bn = nn.InstanceNorm2d(8)  # Image에서는 2d batchnorm이 사용됩니다
        self.pool0 = nn.MaxPool2d(2)
        self.fc = nn.Linear(8*6*6, 10) # Layer 2 (왜 input이 8 * 11 * 11 일까요?)

    def forward(self, x):
        x = self.conv0(x)
        x = self.conv0_bn(x)
        x = F.relu(x)
        x = self.pool0(x)
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        return x

In [88]:
mnist_net = MNIST_Net() # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.Adam(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [89]:
trainer = Trainer(trainloader = trainDataLoader,
                  testloader = testDataLoader,
                  net = mnist_net,
                  criterion = criterion,
                  optimizer = optimizer)

In [90]:
trainer.train(epoch = 4)

[1,   500] loss: 0.589
[1,  1000] loss: 0.221
[1,  1500] loss: 0.170
[2,   500] loss: 0.125
[2,  1000] loss: 0.115
[2,  1500] loss: 0.100
[3,   500] loss: 0.091
[3,  1000] loss: 0.090
[3,  1500] loss: 0.079
[4,   500] loss: 0.074
[4,  1000] loss: 0.078
[4,  1500] loss: 0.069
Finished Training


In [91]:
trainer.test()


Test set:  Accuracy: 9808/10000 (98%)



### (13) 3-Layer Network (Conv+Pool+Fc) + ReLU + Adam + Layer-Norm

- Input: (28 * 28)
- Conv: 8 (7 * 7) filter with stride=2 
- Pool: 2 * 2
- Hidden dimension: 8 * 6 * 6
- Output dimension: 10
- activation: relu
- normalization: Layer-norm
- Optimizer: Adam
- Loss: Cross-Entropy

In [165]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__()
        self.conv0 = nn.Conv2d(in_channels = 1,
                               out_channels = 8,
                               kernel_size = 6,
                               stride = 2) # Layer 1
        self.conv0_bn = nn.LayerNorm([8,12,12])  # Image에서는 2d batchnorm이 사용됩니다
        self.pool0 = nn.MaxPool2d(2)
        self.fc = nn.Linear(8*6*6, 10) # Layer 2 (왜 input이 8 * 11 * 11 일까요?)

    def forward(self, x):
        x = self.conv0(x)
        x = self.conv0_bn(x)

        x = F.relu(x)
        x = self.pool0(x)
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        return x

In [166]:
mnist_net = MNIST_Net() # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.Adam(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [167]:
trainer = Trainer(trainloader = trainDataLoader,
                  testloader = testDataLoader,
                  net = mnist_net,
                  criterion = criterion,
                  optimizer = optimizer)

In [170]:
trainer.train(epoch = 6)

[1,   500] loss: 0.054
[1,  1000] loss: 0.054
[1,  1500] loss: 0.052
[2,   500] loss: 0.044
[2,  1000] loss: 0.049
[2,  1500] loss: 0.053
[3,   500] loss: 0.046
[3,  1000] loss: 0.044
[3,  1500] loss: 0.049
[4,   500] loss: 0.044
[4,  1000] loss: 0.043
[4,  1500] loss: 0.044
[5,   500] loss: 0.041
[5,  1000] loss: 0.037
[5,  1500] loss: 0.043
[6,   500] loss: 0.038
[6,  1000] loss: 0.040
[6,  1500] loss: 0.041
Finished Training


In [171]:
trainer.test()


Test set:  Accuracy: 9827/10000 (98%)



### (14) 3-Layer Network (Conv+Pool+Fc) + ReLU + Adam + Grouph-Norm

- Input: (28 * 28)
- Conv: 8 (7 * 7) filter with stride=2 
- Pool: 2 * 2
- Hidden dimension: 8 * 6 * 6
- Output dimension: 10
- activation: relu
- normalization: Group-norm
- Optimizer: Adam
- Loss: Cross-Entropy

In [179]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__()
        self.conv0 = nn.Conv2d(in_channels = 1,
                               out_channels = 8,
                               kernel_size = 6,
                               stride = 2) # Layer 1
        self.conv0_bn = nn.GroupNorm(num_groups = 4, num_channels = 8)  # Image에서는 2d batchnorm이 사용됩니다
        self.pool0 = nn.MaxPool2d(2)
        self.fc = nn.Linear(8*6*6,10) # Layer 2 (왜 input이 8 * 11 * 11 일까요?)

    def forward(self, x):
        x = self.conv0(x)
        x = self.conv0_bn(x)
        x = F.relu(x)
        x = self.pool0(x)
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        return x

In [180]:
mnist_net = MNIST_Net() # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.Adam(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [181]:
trainer = Trainer(trainloader = trainDataLoader,
                  testloader = testDataLoader,
                  net = mnist_net,
                  criterion = criterion,
                  optimizer = optimizer)

In [182]:
trainer.train(epoch = 8)

[1,   500] loss: 0.534
[1,  1000] loss: 0.192
[1,  1500] loss: 0.137
[2,   500] loss: 0.102
[2,  1000] loss: 0.100
[2,  1500] loss: 0.087
[3,   500] loss: 0.077
[3,  1000] loss: 0.077
[3,  1500] loss: 0.073
[4,   500] loss: 0.069
[4,  1000] loss: 0.069
[4,  1500] loss: 0.061
[5,   500] loss: 0.060
[5,  1000] loss: 0.062
[5,  1500] loss: 0.066
[6,   500] loss: 0.055
[6,  1000] loss: 0.062
[6,  1500] loss: 0.052
[7,   500] loss: 0.053
[7,  1000] loss: 0.058
[7,  1500] loss: 0.054
[8,   500] loss: 0.051
[8,  1000] loss: 0.051
[8,  1500] loss: 0.051
Finished Training


In [183]:
trainer.test()


Test set:  Accuracy: 9797/10000 (98%)



In [125]:
count_parameters(mnist_net)

3202

## Let's Do It: 10000개 이하의 파라미터로 99%의 성능을 달성해 볼까요?

In [104]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__()

    def forward(self, x):
        return x

In [106]:
mnist_net = MNIST_Net() # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.Adam(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

ValueError: optimizer got an empty parameter list

In [107]:
train_network(mnist_net,optimizer,trainDataLoader)

NameError: name 'train_network' is not defined

In [ ]:
test(mnist_net,testDataLoader)